In [1]:
import csv
import pandas as pd

In [2]:
f = open("DataExplorerExport4.csv", "rt")
reader = csv.reader(f)

# i = 0
# for r in reader:
#     print(i, r)
#     i+=1
#     if i==250:
#         break


In [3]:
df = pd.read_csv("DataExplorerExport4.csv")
df.head()

,QuerySetId,Title,Description,QueryBody,CreationDate,Views,Votes
0,14342,What's my Pundit badge progress?,How many of my comments have scores of 5 and h...,-- What's my Pundit badge progress?\n-- How ma...,2010-11-17 09:20:44Z,41810,30
1,541386,NaN,NaN,-- Enter Query Title\n-- Enter Query Descripti...,2016-09-19 09:16:04Z,10,0
2,227856,NaN,NaN,-- Enter Query Title\n-- Enter Query Descripti...,2014-09-25 17:49:53Z,95,0
3,882963,NaN,NaN,-- Enter Query Title\n-- Enter Query Descripti...,2018-08-04 18:05:24Z,12,0
4,1329429,NaN,NaN,select count (*) from posts,2020-11-12 00:51:16Z,13,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1574098 entries, 0 to 1574097
Data columns (total 7 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   QuerySetId    1574098 non-null  int64 
 1   Title         498597 non-null   object
 2   Description   100955 non-null   object
 3   QueryBody     1574098 non-null  object
 4   CreationDate  1574098 non-null  object
 5   Views         1574098 non-null  int64 
 6   Votes         1574098 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 84.1+ MB


In [6]:
df.iloc[170]

QuerySetId                   1336634
Title                            NaN
Description                      NaN
QueryBody       select *\nfrom votes
CreationDate    2020-11-25 18:10:37Z
Views                              4
Votes                              0
Name: 170, dtype: object

In [7]:
df['QueryBody'].nunique()

1142485

In [8]:
df['Title'].nunique()

35673

In [24]:
new_df = df
new_df['Title'] = new_df['Title'].str.strip()
print(new_df['Title'].nunique())
new_df = new_df[~new_df['Title'].str.contains("SELECT", na=False, case=False)]
new_df = new_df[~new_df['Title'].str.contains("CREATE", na=False, case=False)]
new_df = new_df[~new_df['Title'].str.contains("DECLARE", na=False, case=False)]
print(new_df['Title'].nunique())
new_df = new_df[new_df['QueryBody'].str.len() < 800]
print(new_df['Title'].nunique())
new_df = new_df[new_df['Title'].str.count(' ') > 3]
print(new_df['Title'].nunique())

35499
34418
30721
14696


In [13]:
new_df = new_df.drop_duplicates(subset=['QueryBody'])
new_df.sort_values(by=['Title','CreationDate'], inplace=True)

In [22]:
new_df.head()

,QuerySetId,Title,Description,QueryBody,CreationDate,Views,Votes
7,1116781,TOP 10,NaN,SELECT OwnerUserId\nFROM Posts,2019-09-30 15:03:18Z,23,0
9,139893,Questions by tags,NaN,Select * from posttags,2013-09-30 04:27:43Z,3510,0
13,14542,Nick Craver comparison.,What questions does Nick Craver have a higher/...,-- Nick Craver comparison.\n-- What questions ...,2010-11-20 20:34:20Z,2522,1
21,14772,"""Closed"" questions",Finds all questions that appear to be closed b...,"-- ""Closed"" questions\n-- Finds all questions ...",2010-11-24 04:37:01Z,3484,0
28,1157464,Query1,NaN,select top (10) * from Posts,2019-11-21 00:49:53Z,32,0


In [15]:
def is_valid(title, query):
    title = title.lower()
    query = query.lower()
    title_tokens = title.split()
    query_tokens = query.split()
    
    def valid_numbers():
        title_numbers = [t for t in title_tokens if t.isdigit()]
        query_numbers = set([t for t in query_tokens if t.isdigit()])
        return all([n in query_numbers for n in title_numbers])
    
    def valid_aggregators():
        if "number of" in title:
            if "count" not in query:
                return False
        if "average" in title:
            if "avg" not in query and "/" not in query:
                return False
        if any([s in title for s in ["by month","by year","by tag","by country","each month","each year","each tag","each country"]]):
            if "group by" not in query:
                return False
        return True
    
    check = [valid_numbers, valid_aggregators]
    return all([c() for c in check])

print(is_valid("there are 300 numbers 100 10", "query with 300 100 10"))
print(is_valid("there are 300 numbers 100 10", "query with 300"))
print(is_valid("there are 300 numbers 100 10", "query with"))

print(is_valid("number of something", "query with"))
print(is_valid("number of something", "query with count"))

print(is_valid("Best answer degree by country", "query with count"))
print(is_valid("Best answer degree by country", "query with count group by"))


True
False
False
False
True
False
True


In [17]:
new_df['valid'] = [is_valid(title, query) for title, query in zip(new_df['Title'], new_df['QueryBody'])]

In [18]:
print(new_df.size)
print(new_df['Title'].nunique())
print((new_df['valid'] == True).sum())

1020648
14320
109539


In [19]:
new_df[:20000].to_csv('first20000.csv',escapechar="\\")